<a href="https://colab.research.google.com/github/wandb/wandb-workspaces/blob/example-notebook/Programmatic_Workspace_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Sample Notebook for [Weights & Biases Programmatic Workspace](https://docs.wandb.ai/guides/app/pages/workspaces#saved-workspace-views)


### Overview
* [wandb-workspaces](https://github.com/wandb/wandb-workspaces/tree/main) is a Python library for programmatically working with Weights & Biases workspaces and Reports. You can define a workspace programmatically by defining the workspace's properties, such as its name, associated entity and project, and the sections it should contain.

* Programmatically create and customize workspaces by defining configurations, setting panel layouts, and organizing sections. Load and modify workspaces via URL, filter and group runs using expressions, and customize run appearances. Easily copy views across workspaces for integration.

### Known limitations
* Only compatible with Saved Views (URLs ending with ?nw=...). Clone a `Personal Workspace` as `Saved Views` for programmatic interaction.

### Notebook Instructions:

* Run each cell step by step. The notebook will log sample data to a project named `workspace-api-example`.
* The notebook will create the project, log sample a to it, and then demonstrate the workspace examples using this data.

# Install and Import Dependencies

In [ ]:
# Install dependencies

!pip install wandb wandb-workspaces
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go

In [ ]:
# makes formatting nicer (you might want to use dark mode on colab)
%load_ext rich

In [ ]:
# Import dependencies

import os
import wandb
import wandb_workspaces.workspaces as ws
import wandb_workspaces.reports.v2 as wr # We use the Reports API panels

# Create a Project to interact with programmatically

In [ ]:
# Initialize WandB
wandb.login()

# Function to create a new project and log sample data
def create_project_and_log_data():
    project = "workspace-api-example"  # Default project name

    # Initialize a run to log some sample data
    with wandb.init(project=project, name="sample_run") as run:
        for step in range(100):
            wandb.log({
                "Step": step,
                "val_loss": 1.0 / (step + 1),
                "val_accuracy": step / 100.0,
                "train_loss": 1.0 / (step + 2),
                "train_accuracy": step / 110.0,
                "f1_score": step / 100.0,
                "recall": step / 120.0,
            })
    return project

# Create a new project and log data
project = create_project_and_log_data()
entity = wandb.Api().default_entity

# Summary of Programmatic Workspace features:
The Weights & Biases Workspace API allows you to create, manage, and customize workspaces programmatically. These capabilities are designed to help you organize and visualize your machine learning experiments more effectively.


### 1. Workspace Creation
* **Define Create Workspaces:** You can create a workspace with a specific
name, associate it with a project, and organize it into sections.
* **Customization:** Customize each workspace with settings such as axis configurations, smoothing types, outlier handling, and more.

### 2. Sections and Panels
* **Sections:** Organize your workspace into multiple sections, each containing a specific set of panels. Sections can be opened or closed based on your preference.
* **Panels:** Add various types of panels such as line plots, bar plots, and scalar charts to visualize different metrics.
* **Layouts:** Customize the layout of panels within each section using standard or custom grid layouts.

### 3. Workspace Settings
* **Global Settings:** Configure global settings for the workspace, including axis settings, smoothing options, outlier handling, and tooltip settings.
* **Section and Panel Settings:** Override global settings at the section or panel level to provide more granular control over the visualization.

### 4. Run Management
* **Run Sets:** Manage sets of runs in the workspace. Apply filters to include or exclude specific runs based on metrics or metadata.
* **Grouping and Ordering:** Group and order runs within the run set to better analyze and compare different experiments.
* **Run Customization:** Customize individual run settings such as color and visibility within the workspace..

### 5. Filters and Queries
* **Filter Expressions:** Apply complex filter expressions to include or exclude runs based on specific criteria.
* **Regex Queries:** Use regular expressions to search and filter runs within the workspace.

### 6. Saving and Loading Workspaces
* **Save Workspaces:** Save the current state of a workspace, making it accessible as a `Saved view` from the Project.
* **Load Workspaces:** Load an existing workspace from a URL to review or modify it.
* **Clone Workspaces:** Save a modified workspace as a new view, allowing you to create multiple versions of a workspace without affecting the original.

In [ ]:
# Example 00: Creating a New Workspace with Sections and Panels
def example_00(entity, project):
    # Declare the workspace all at once
    workspace = ws.Workspace(
        name="Example W&B Workspace",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Validation Metrics",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.BarPlot(metrics=["val_accuracy"]),
                    wr.ScalarChart(metric="f1_score", groupby_aggfunc="mean"),
                ],
                is_open=True,
            ),
        ],
    )

    # Save the workspace to WandB
    workspace.save()
    print("Sample Workspace 00 saved.")
    return workspace.url

# Run example 00 to create a workspace and get its URL
workspace_url = example_00(entity, project)

# Example 01: Loading and Cloning a Workspace
def example_01(url):
    # Load the workspace from the provided URL
    workspace = ws.Workspace.from_url(url)

    # Edit the workspace name and save it
    workspace.name = "Updated Workspace Name"
    workspace.save()
    print("Workspace 01 updated and saved.")

    # Save the workspace as a new view
    new_url = workspace.save_as_new_view()
    print(f"Workspace 01 saved as new view. New URL: {new_url}")

# Run example 01 with the URL from example 00
example_01(workspace_url)

# Example 02: Basic Workspace Settings
def example_02(entity, project):
    workspace = ws.Workspace(name="An example workspace", entity=entity, project=project)

    # Add sections with panels to the workspace
    workspace.sections = [
        ws.Section(
            name="Validation",
            panels=[
                wr.LinePlot(x="Step", y=["val_loss"]),
                wr.LinePlot(x="Step", y=["val_accuracy"]),
                wr.ScalarChart(metric="f1_score", groupby_aggfunc="mean"),
                wr.ScalarChart(metric="recall", groupby_aggfunc="mean"),
            ],
            is_open=True,
        ),
        ws.Section(
            name="Training",
            panels=[
                wr.LinePlot(x="Step", y=["train_loss"]),
                wr.LinePlot(x="Step", y=["train_accuracy"]),
            ],
        ),
    ]

    # Configure settings on the workspace
    workspace.settings = ws.WorkspaceSettings(
        x_min=0,
        x_max=75,
        smoothing_type="gaussian",
        point_visualization_method="bucketing",
    )

    # Configure settings on individual sections
    section = workspace.sections[0]
    section.panel_settings = ws.SectionPanelSettings(
        x_min=25,
        x_max=50,
        smoothing_type="none",
    )

    # Configure settings on individual panels
    panel = section.panels[0]
    panel.title = "Validation Loss Custom Title"
    panel.title_x = "Custom x-axis title"

    # Save the workspace
    workspace.save()
    print("Workspace 02 with custom settings saved.")

# Example 03: Working with Filters
def example_03(entity, project):
    # Create filters using basic metrics and python expressions
    val_loss_filter = ws.Metric("val_loss") < 1

    # Create filters using UI references like Name, Tags, or ID
    run_name_filter = ws.Metric("Name") == "sample_run"
    run_id_filter = ws.Metric("ID").isin([run.id for run in wandb.Api().runs(f"{entity}/{project}")])

    # Create a workspace with filters
    workspace = ws.Workspace(
        name="An example workspace using filters",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Validation",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.LinePlot(x="Step", y=["val_accuracy"]),
                ],
                is_open=True,
            ),
        ],
        runset_settings=ws.RunsetSettings(
            filters=[
                ws.Metric("Name") == "sample_run",
                ws.Metric("ID").isin([run.id for run in wandb.Api().runs(f"{entity}/{project}")]),
            ]
        ),
    )

    # Save the workspace
    workspace.save()
    print("Workspace 03 with filters saved.")


In [ ]:
# Run example 00: Creating a New Workspace with Sections and Panels

# Purpose: Demonstrates how to create a new workspace and populate it with sections and panels.
# Benefit: Helps users organize and visualize their project metrics in a structured manner.

example_00(entity,project)

In [ ]:
# Run example 01: Loading and Cloning a Workspace

# Purpose: Demonstrates how to load an existing workspace, modify it, and save it as a new view.
# Benefit: Allows users to duplicate and customize workspaces without affecting the original setup.
example_01(workspace_url)


In [ ]:
# Run example 02: Basic Workspace Settings

# Purpose: Demonstrates how to create a workspace, add sections with panels, and configure settings for the workspace and individual panels.
# Benefit: Provides users with the ability to customize their workspace layout and visualization settings.

example_02(entity, project)


In [ ]:
# Run example 03: Working with Filters

# Purpose: Demonstrates how to create a workspace using filters to manage runs based on specific criteria.
# Benefit: Allows users to focus on specific subsets of their data, making it easier to analyze and compare results.

example_03(entity, project)